
## Task 1: Binary Representations

Task 1 is showing how 4 different functions are used to manipulate **32-bit numbers**.  
These functions are mainly used in **encryption methods.** 


### Rotating Bits Left (`rotl`)

The `rotl(x, n)` function moves the bits of a number to the left by **n** places.  
When the numbers have reached the end of the 32-bit on the left it then wraps around to the right side to continue.   
**0xFFFFFFFF** is used to keep the number within the 32 bits. [Rotating bits of a number](https://www.geeksforgeeks.org/python3-program-to-rotate-bits-of-a-number/).

In [41]:
def rotl(x, n=1):
    """
    Rotates the bits in a 32-bit integer to the left by n places.
    """
    n = n % 32  # Ensure n is within valid bit range (0-31)
    return ((x << n) & 0xFFFFFFFF) | (x >> (32 - n))

#### **Rotating Bits Right (`rotr`)**

The `rotr(x, n)` function moves the bits of a number to the left by **n** places.  
When the numbers have reached the end of the 32-bit on the right it then wraps around to the left side to continue.  
Commonly used in **cryptography** for a fast way to secure data.  

In [42]:
def rotr(x, n=1):
    """
    Rotates the bits in a 32-bit integer to the right by n places.
    """
    n = n % 32  # Ensure n is within valid bit range (0-31)
    return (x >> n) | ((x << (32 - n)) & 0xFFFFFFFF)

#### **Bitwise Choice (`ch`)**

The `ch(x, y, z)` function chooses bits from `y` or `z` based on the value in `x`.  
If a bit in `x` is `1`, it takes the bit from `y`.  
If a bit in `x` is `0`, it takes the bit from `z`.  
`ch` is important in **cryptography**, especially in SHA-256 hashing, where bits are choosen based on conditions.  

In [43]:
def ch(x, y, z):
    """
    Chooses bits from y where x has bits set to 1, and from z where x has bits set to 0.
    Returns:
    int: Resulting 32-bit integer after bitwise choice
    """
    return (x & y) | (~x & z)  # If x bit is 1 -> choose from y, else from z


#### **Bitwise Majority (`maj`)**

The `maj(x, y, z)` function checks each bit position in `x`,`y` and `z` and then chooses to keep the **majority value**.  
If at least **two out of the three** numbers have a `1` at a bit position, the result will also have `1` there.  
Otherwise, it will be `0`.  
This function is used in **secure hashing algorithms** to ensure consistency.  

In [44]:
def maj(x, y, z):
    """
    Majority votes of bits in x, y, and z.

    Parameters:
    x (int): 32-bit integer
    y (int): 32-bit integer
    z (int): 32-bit integer

    Returns:
    int: Resulting 32-bit integer after bitwise majority
    """
    return (x & y) | (x & z) | (y & z)  # A bit is 1 if at least two of x, y, z have 1s

### Testing the Bitwise Functions

Here we define 3 **32-bit integers** and showcase the functions for testing.

#### **Defining 32-bit Test Values**
We use 3 **binary numbers** as inputs:
- `x = 0b10110011100011110000111100001111` → A randomly chosen **32-bit integer**.
- `y = 0b11001100110011001100110011001100` → A pattern of alternating bits.
- `z = 0b00001111000011110000111100001111` → High and low bit sequences.

These values showcase to us that the functions correctly handle the bits at different positions.

In [45]:
# Define 32-bit example values for testing
x = 0b10110011100011110000111100001111  # Example 32-bit integer
y = 0b11001100110011001100110011001100
z = 0b00001111000011110000111100001111

# Testing the functions
if __name__ == "__main__":
    print(f"Original x: {bin(x)}")
    print(f"rotl(x, 4): {bin(rotl(x, 4))}")
    print(f"rotr(x, 4): {bin(rotr(x, 4))}")
    print(f"ch(x, y, z): {bin(ch(x, y, z))}")
    print(f"maj(x, y, z): {bin(maj(x, y, z))}")

Original x: 0b10110011100011110000111100001111
rotl(x, 4): 0b111000111100001111000011111011
rotr(x, 4): 0b11111011001110001111000011110000
ch(x, y, z): 0b10001100100011000000110000001100
maj(x, y, z): 0b10001111100011110000111100001111


## Task 2: Hash Functions

Task 2 function coverts `(s)` a string into a numeric hash value.  
`hashval = 0` is the first value.  
For loop which iterates through each char in `(s)`.  
`(ord(char))` is used to convert the char to ASCII.  
Multiply the hash value by `31` and then add the char ASCII value.  

In [46]:
def hash_function(s: str) -> int:
    """
    Parameters:
    s (str): The input string.

    Returns:
    int: Hash value mod 101.
    """
    hashval = 0
    for char in s:
        hashval = ord(char) + 31 * hashval
    return hashval % 101

### Testing the Hash Function

Define a list of words to hash.  
Call hash_function() on each word.  
Print the results.  

In [53]:
# Testing the function
test_strings = ["john", "smith", "computational", "theory"]
for string in test_strings:
    print(f"Hash of '{string}': {hash_function(string)}")

Hash of 'john': 97
Hash of 'smith': 19
Hash of 'computational': 42
Hash of 'theory': 77


**Why Use 31 and 101?**

## Task 3: SHA256

In this step, we **read the input file** in **binary mode** and store its contents in a variable.  
We also calculate the **original message length in bits** and initialize global variables.

- The file is opened in **binary mode** (`rb`).
- The data is read and stored in `data`.
- The original message length is calculated in **bits**.
- If the file is **missing**, an error message is shown.

In [ ]:
import os  # Importing OS for file handling

# Define file path
file_path = "test.txt"

try:
    with open(file_path, "rb") as f:
        data = f.read()  # Read file contents as bytes
except FileNotFoundError:
    print(f"Error: File '{file_path}' not found.")
    
# Store variables globally so they can be used in later cells
original_bit_length = len(data) * 8  

print("Step 1: File read successfully!")


## Step 2: Append '1' Bit

SHA-256 padding starts with **appending a single '1' bit** (`0x80` in hex).  
This ensures that the message **always begins padding with `10000000`** in binary.

📌 **What Happens Here?**
- `b'\x80'` is added to the `data`, creating the initial padded message.
- This ensures padding starts with a `1` followed by zeros.

In [ ]:
# Append the '1' bit (0x80 in hex) to the message
padded_data = data + b'\x80'

print("Step 2: 1-bit appended.")

## Task 4: Prime Numbers


## Task 5: Roots

## Task 6: Proof of Work

## Task 7: Turing Machines

## Task 8: Computational Complexity